# Demonstrate Seq2Seq Wrapper with Cornell Movie Dialog Corpus

In [153]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.cornell_corpus import data
import data_utils

In [154]:
import importlib
importlib.reload(data)



<module 'datasets.cornell_corpus.data' from '/Users/rashmisharma/Documents/GitHub/sqschatbot/practical_seq2seq-master/datasets/cornell_corpus/data.py'>

In [155]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

print(trainX[0:5])

print(trainX.shape[-1])
print(trainY.shape[-1])

[[  52   22  114   17  898    1    1   11 3984 7674   28  410   81 3714
     1 1261  501   55   29    4    1  183    0    0    0]
 [  24    4    1   11    1   11 7953  411  155    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  36  573   12   44   49   42 1051   80   23  194  183    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [1868   46  131   22   96  154 4250    6 1194    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [   1  892    1   17   14   25  371    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
25
25


In [156]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [157]:
import seq2seq_wrapper

In [158]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/cornell_corpus/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [ ]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 32)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [ ]:
sess = model.restore_last_session()
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>


In [ ]:
input_ = test_batch_gen.__next__()[0]
print(input_)
print(input_[0])
output = model.predict(sess, input_[0:25])
print(output.shape)
print(input_[0].shape)
print(input_[0])

In [ ]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

In [ ]:
print(input_[0])

In [ ]:
def process_test_data(question):

    metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')

    w_2_id = dict(metadata['w2idx'])
    id_2_w = metadata['idx2w']

    sent = 'hello, what is your name?'
   
    questions = sent.lower()
    qtokenized =  [w.strip() for w in questions.split(' ') if w]
    input_array = np.array(data.pad_seq(qtokenized, w_2_id, 30))

    print(input_array)


In [91]:
process_test_data("df")

[ 1  8 14 23  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]


In [151]:
import nltk as nltk
import seq2seq_wrapper
def chat(model,sess,train_seq_len,y_seq_len):
    # load data from pickle and npy files
    metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')

    w_2_id = dict(metadata['w2idx'])
    id_2_w = metadata['idx2w']

    sent = 'hello, what is your name?'
   
    questions = sent.lower()
    qtokenized =  [w.strip() for w in questions.split(' ') if w]
    input_array = np.array([data.pad_seq(qtokenized, w_2_id, 25)])
    input_array = input_array.T
    #print(input_array)
    emb_dim = 400

    batch_size = 16
    xvocab_size = len(metadata['idx2w'])  
    yvocab_size = xvocab_size
    emb_dim = 1024
    
    #print(input_array)
    
    model = seq2seq_wrapper.Seq2Seq(xseq_len=train_seq_len,
                               yseq_len=y_seq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/cornell_corpus/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )
    sess = model.restore_last_session()
    
    predict = model.predict(sess=sess, X=input_array)

    print(predict)
    response = [id_2_w[i] for i in predict[0]]
    print(response)

In [152]:
chat(model,sess,25,25)

[[ 1]
 [ 8]
 [14]
 [23]
 [ 1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
[[ 1]
 [ 8]
 [14]
 [23]
 [ 1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
<log> Building Graph 

Exception ignored in: <bound method TF_Buffer.<lambda> of <tensorflow.python.pywrap_tensorflow.TF_Buffer; proxy of <Swig Object of type 'TF_Buffer *' at 0x133cd9ed0> >>
Traceback (most recent call last):
  File "/Users/rashmisharma/anaconda2/envs/tensorflow1.0/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 339, in <lambda>
    __del__ = lambda self: None

</log>


KeyboardInterrupt


FailedPreconditionError: Attempting to use uninitialized value decoder/embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/output_projection_wrapper/biases
	 [[Node: decoder/embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/output_projection_wrapper/biases/read = Identity[T=DT_FLOAT, _class=["loc:@decoder/embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/output_projection_wrapper/biases"], _device="/job:localhost/replica:0/task:0/cpu:0"](decoder/embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/output_projection_wrapper/biases)]]

Caused by op 'decoder/embedding_rnn_seq2seq/embedding_rnn_decoder/rnn_decoder/output_projection_wrapper/biases/read', defined at:
  File "/Users/rashmisharma/anaconda2/envs/tensorflow1.0/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/rashmisharma/anaconda2/envs/tensorflow1.0/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/rashmisharma/anaconda2/